In [83]:
import logging
import argparse
import sys
import warnings
import importlib
import json
import pprint
import re
import requests

import pandas as pd
import ocdskit
import ijson
from ocdskit.exceptions import CommandError
from ocdskit.cli.commands import compile as ocds_compile
from ocdskit.combine import merge
from pydantic import BaseModel, HttpUrl

import scraping_by.scrapers.pcs.response_model as dm
from scraping_by.scrapers.contract import Contract, ContractInterface
from scraping_by.scrapers.pcs.interface import PCSRelease

In [84]:
response = requests.get("http://localhost:8000/contracts/pcs")

In [90]:
df_contracts = pd.concat([pd.Series(c) for c in response.json()], axis=1).T

In [91]:
keywords = {
    "\sAI-?\s",
    "data",
    "software",
    "\smachine\slearning\s",
    "\sIT\s"
}
pattern = r"|".join(keywords)

print("Searching with pattern:")
print(pattern)

matches = (
    df_contracts["tender_description"].str.contains(pattern, flags=re.IGNORECASE) |
    df_contracts["notice_description"].str.contains(pattern, flags=re.IGNORECASE) |
    df_contracts["lots_description"].str.contains(pattern, flags=re.IGNORECASE)
)
df_matches = df_contracts[matches]
print(df_matches.shape[0], "total matches")

Searching with pattern:
\sIT\s|\sAI-?\s|data|software|\smachine\slearning\s
17 total matches


In [82]:
for ix, row in df_matches.iterrows():
    print("\n*\n")
    print(ix, row.notice_title, row.value, row.first_lot_duration_days, end="\n"*2)
    print(row.lots_description, end="\n"*2)
    print(row.notice_url)


*

0 CT0818 Council Records - Storage and Management None 3600

Centralised offsite storage of Council records has been a long-standing requirement, which has become essential over the years due to office rationalisations and increasing compliance requirements around Scottish public sector records management.
 The Council requires new arrangements for its offsite storage and management of its paper records. This is a recurring need and breaks down as follows:
 -	Secure offsite physical storage arrangements for up to 65,000 boxes at the start, but aiming to reduce this significantly over a 10 year period.
 -	Routine and emergency access requirements to boxes and files, both onsite and by delivery to multiple Council office locations via dedicated transport.
 -	Online inventory management solution that manages both boxes and individual files (where required), including audited and authenticated disposal and retrieval processes, legal holds, classification by business function and data s